## Доверительные интервалы. 

In [1]:
import numpy as np 
import pandas as pd
from scipy import stats #чтобы считать квантили

In [2]:
df = pd.read_csv('flats.tsv', sep='\t')

In [3]:
df.head()

,n,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
0,1,81,58,40,6.0,12.5,7,1,1,1,3
1,2,75,44,28,6.0,13.5,7,1,0,1,6
2,3,128,70,42,6.0,14.5,3,1,1,1,3
3,4,95,61,37,6.0,13.5,7,1,0,1,1
4,5,330,104,60,11.0,10.5,7,0,1,1,3


In [4]:
# посмотрим на распределения в таблице
df.describe()

,n,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
count,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000
mean,1020.500000,127.496569,73.084314,46.337255,8.898529,11.015686,8.117157,0.685784,0.323039,0.790686,4.322059
std,589.041594,51.878220,15.123450,7.894348,2.787073,3.375539,3.815574,0.464317,0.467752,0.406918,2.183289
min,1.000000,50.000000,44.000000,28.000000,5.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,510.750000,95.000000,62.000000,42.000000,7.000000,9.000000,5.000000,0.000000,0.000000,1.000000,3.000000
50%,1020.500000,115.000000,73.500000,45.000000,9.000000,12.000000,7.000000,1.000000,0.000000,1.000000,4.000000
75%,1530.250000,142.000000,79.000000,50.000000,10.000000,13.500000,10.000000,1.000000,1.000000,1.000000,6.000000
max,2040.000000,730.000000,192.000000,102.000000,25.000000,17.000000,20.000000,1.000000,1.000000,1.000000,8.000000


In [6]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

# посмотрим на распределение цен

fig = px.histogram(df, x="price")
fig.show()

In [7]:
df['price_log'] = df['price'].apply(lambda x: np.log(x))
fig = px.histogram(df, x="price_log")
fig.show()

In [8]:
stats.kstest(df['price'], 'norm',  args=(df.price.mean(), df.price.std(ddof=1)))

KstestResult(statistic=np.float64(0.1508501598050489), pvalue=np.float64(5.415515510735646e-41), statistic_location=np.int64(140), statistic_sign=np.int8(1))

In [9]:
stats.kstest(df['price_log'], 'norm',  args=(df['price_log'].mean(), df['price_log'].std(ddof=1)))

KstestResult(statistic=np.float64(0.07697286130504083), pvalue=np.float64(5.856877429594e-11), statistic_location=np.float64(4.787491742782046), statistic_sign=np.int8(1))

## Построим 95% доверительный интервал для средней цены квартиры  с помощью t-распределения

Напомним, что доверительный интервал вычисляется следующим образом: 

$$
 \bar{x} - {t^*}{\sqrt{\frac{\bar{s}^2}{n}}} < \mu < \bar{x} + {t^*}{\sqrt{\frac{\bar{s}^2}{n}}} 
$$


In [10]:
alpha = 0.05


n = df.price.count()
sample_mean = df.price.mean()
sample_var = df.price.var()

**ppf**: percent point function (or inverse cumulative distribution function) ppf returns the value x of the variable that has a given cumulative distribution probability (cdf). Thus, given the cdf(x) of a x value, ppf returns the value x itself, therefore, operating as the inverse of cdf.

In [14]:
# Code here

t = stats.t(n-1).ppf(1-alpha/2)

t_left = sample_mean - t*np.sqrt(sample_var/n)
t_right = sample_mean + t*np.sqrt(sample_var/n)

print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(t_left, t_right, t_right - t_left))


Доверительный интервал [125.2; 129.7] ширины 4.505


То же самое можно сделать из пакета

In [15]:
stats.t.interval(0.95, n-1, sample_mean, np.sqrt(sample_var/n))

(np.float64(125.24401082202615), np.float64(129.7491264328758))

## Cравним цены в разных районах

Построим $95\%$ асимптотический доверительный интервал для разницы в средней стоимости квартир в монолитных и панельных домах.

$$
\bar x - \bar y \pm t_{crit} \cdot \sqrt{\frac{\hat\sigma_x^2}{n_x} + \frac{\hat\sigma_y^2}{n_y}}
$$

район - code – число от 1 до 8, при помощи которого мы группируем наблюдения по
подвыборкам:
1. Наблюдения сгруппированы на севере, вокруг Калужско-Рижской линии
метрополитена
2. Север, вокруг Серпуховско-Тимирязевской линии метрополитена
3. Северо-запад, вокруг Замоскворецкой линии метрополитена
4. Северо-запад, вокруг Таганско-Краснопресненской линии метрополитена
5. Юго-восток, вокруг Люблинской линии метрополитена
6. Юго-восток, вокруг Таганско-Краснопресненской линии метрополитена
7. Восток, вокруг Калиниской линии метрополитена
8. Восток, вокруг Арбатско-Покровской линии метрополитена

In [16]:
df.groupby('code').agg({'price':'mean'}).sort_values('price')

,price
code,
6,109.964981
2,110.316279
7,114.230088
5,115.779762
1,134.411765
8,136.744444
3,148.246377
4,148.693780


In [17]:
x, y = df[df['code']==6]['price'], df[df['code']==8]['price']

In [18]:
diff = x.mean() - y.mean()
n_x, n_y = len(x), len(y)

diff_sd = np.sqrt(x.var()/n_x + y.var()/n_y)

In [19]:
stats.t.interval(0.95, n-1, loc=diff, scale=diff_sd)

(np.float64(-34.95399450068699), np.float64(-18.60493329870774))

In [20]:
left, right = stats.t.interval(0.95, n-1, loc=diff, scale=diff_sd)
print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(left, right, (right-left)))

Доверительный интервал [-34.95; -18.6] ширины 16.35
